In [1]:
from matplotlib import pyplot as plt
import numpy as np

plt.style.use('ggplot')

In [2]:
# Analysis of 3D rotation fitting. A computational method of hip joint center

def TransformationMatrix(bodyVecs, globalVecs, weights):
    n = bodyVecs.shape[1]
    Pmean = np.mean(bodyVecs, axis=1)
    Qmean = np.mean(globalVecs, axis=1)
    
    P = bodyVecs - np.tile(Pmean.reshape(3,-1), (1,n))
    Q = globalVecs - np.tile(Qmean.reshape(3,-1), (1,n))
    
    K = np.zeros((3, 3))
    for i in range(n):
        K += weights[i] * np.matmul(Q[:,i].reshape(3,1), P[:,i].reshape(1,3))
    
    u,s,vh = np.linalg.svd(K)
    
    vin = np.identity(3)
    vin[2,2] = np.linalg.det(np.matmul(u,vh))
    
    R = np.matmul(u, np.matmul(vin, vh))
    d = Qmean-np.dot(R,Pmean)
    
    transformMat = np.identity(4)
    transformMat[0:3, 0:3] = R
    transformMat[0:3, 3] = d
    
    return transformMat

In [3]:
class fileReader:
    def __init__(self, fileName, columnLabelsLine=5, numHeaderLines=8, delimiter=","):
        self.Data = np.genfromtxt(fileName, delimiter=delimiter, skip_header=numHeaderLines)
        self.NumFrames = self.Data.shape[0]
        #self.NumFrames = 20000
        with open(fileName) as infile:
            content = infile.readlines()
            row = content[columnLabelsLine]
        self.dataIdx = {label:i for i,label in enumerate(row.split(delimiter)) if label != ""}
    
    def getDataForMarkers(self, nLine, markers):           
        result = np.zeros((3, len(markers)))
        for col, marker in enumerate(markers):
            if not marker in markers:
                print("Marker not found")
                return -1
            result[:, col] = self.Data[nLine, self.dataIdx[marker]: self.dataIdx[marker]+3]
        return result

In [4]:
class body:
    def __init__(self, file, markers, weights=None, iterations=3):
        # Computes the local body vecs for the markers
        self._markers = markers
        self._weights = np.ones((len(markers),))
        if (type(weights) == np.ndarray):
            self._weights = weights
        
        self._bodyVecs = file.getDataForMarkers(0, self._markers)        
        transformationMatrices = [None]*file.NumFrames

        for i in range(iterations):
            # Computing the transformation matrixes
            transformationMatrices = self.computeTransformMats(file)
            
            # Updating the local body vecs
            for j in range(len(self._markers)):
                localMarkerVecs = np.ones((4, file.NumFrames))
                for m in range(file.NumFrames):
                    # Inverse Transformation matrix
                    Tinv_t = np.identity(4)
                    Tinv_t[0:3, 0:3] = transformationMatrices[m][0:3, 0:3].T
                    Tinv_t[0:3,3] = -transformationMatrices[m][0:3, 3]

                    globalVec_t = np.ones((4,))
                    globalVec_t[0:3,] = file.getDataForMarkers(m, [self._markers[j]]).reshape(3,)
                    localMarkerVecs[:, m] = np.matmul(Tinv_t, globalVec_t)
                    
                T_t0 = transformationMatrices[0]
                self._bodyVecs[:, j] = np.matmul(T_t0, np.mean(localMarkerVecs, axis=1))[0:3]
        
    def computeTransformMats(self, file):
        transformationMatrices = [None]*file.NumFrames        
        for i in range(file.NumFrames):
            globalVecs = file.getDataForMarkers(i, self._markers)
            transformationMatrices[i] = TransformationMatrix(self._bodyVecs, globalVecs, self._weights)
        return transformationMatrices

In [5]:
def computeFJC(parent_transformMats, child_transformMats):    
    nRows = len(parent_transformMats)
    A = np.zeros((3*nRows, 6))
    b = np.zeros((3*nRows, 1))
    
    for i in range(nRows):
        parent_transformMat = parent_transformMats[i]
        child_transformMat = child_transformMats[i]

        R1 = parent_transformMat[0:3, 0:3]
        R2 = child_transformMat[0:3, 0:3]
        
        d1 = parent_transformMat[0:3,3]
        d2 = child_transformMat[0:3,3]
        
        A[3*i:3*(i+1), :] = np.concatenate((R1, -R2), axis=1);
        b[3*i:3*(i+1), 0] = d2 - d1

    sHJC_lHJC = np.linalg.lstsq(A,b, rcond=-1)[0]

    # Joint Center in Parent fixed frame
    sHJC = np.concatenate((sHJC_lHJC[0:3,0], np.array([1])))

    # Joint Center in Child fixed frame
    lHJC = np.concatenate((sHJC_lHJC[3:,0], np.array([1])))
    
    return sHJC, lHJC

In [6]:
rHjcFileName = "RightStarArc1.csv"
#rAjfileName = "RAJC.csv"
lHjcFileName = "LeftStarArc1.csv"
#lAjfileName = "LAJC.csv"

rightArcFile = fileReader(rHjcFileName)
leftArcFile = fileReader(lHjcFileName)

# Defines the local vecs
print("Computing body vecs")
pelvis = body(rightArcFile, ["RAsis", "LAsis", "RPsi", "LPsi"])
femurR = body(rightArcFile, ["RThigh1", "RThigh2", "RThigh3", "RKneeOut", "RKneeIn"], np.array([1, 1, 1, 0, 0]))
femurL = body(leftArcFile, ["LThigh1", "LThigh2", "LThigh3", "LKneeOut", "LKneeIn"], np.array([1, 1, 1, 0, 0]))

print("Computing joint centers")
sRHJC, lRHJC = computeFJC(pelvis.computeTransformMats(rightArcFile), femurR.computeTransformMats(rightArcFile))
sLHJC, lLHJC = computeFJC(pelvis.computeTransformMats(leftArcFile), femurL.computeTransformMats(leftArcFile))

Computing body vecs
Computing joint centers


In [8]:
tPoseFileName = "TPosePelvis2.csv"
file = fileReader(tPoseFileName)
nRows = file.NumFrames
#nRows = 1
out = np.zeros((nRows, 3*8))

pelvis_transformMats = pelvis.computeTransformMats(file)
femur_r_transformMats = femurR.computeTransformMats(file)
femur_l_transformMats = femurL.computeTransformMats(file)

for i in range(nRows):
    ## Right leg 
    rHJC_global = 0.5*(np.matmul(pelvis_transformMats[i], sRHJC) + np.matmul(femur_r_transformMats[i], lRHJC))
    rKJC_global = np.mean(file.getDataForMarkers(i, ["RKneeOut", "RKneeIn"]), axis=1)
    
    ## Left leg
    lHJC_global = 0.5*(np.matmul(pelvis_transformMats[i], sLHJC) + np.matmul(femur_l_transformMats[i], lLHJC))
    lKJC_global = np.mean(file.getDataForMarkers(i, ["LKneeOut", "LKneeIn"]), axis=1)
    
    vin = lHJC_global-rHJC_global
    print(vin[2])
    
    # Derived markers
    midHJC_global = (rHJC_global+lHJC_global)/2
    midAsis = np.mean(file.getDataForMarkers(i, ["RAsis", "LAsis"]), axis=1)
    midPsis = np.mean(file.getDataForMarkers(i, ["RPsi", "LPsi"]), axis=1)
    midPelvis = (midAsis+midPsis)/2
    
    out[i,:] = np.concatenate((midAsis, midPsis, midPelvis,
                               rHJC_global[0:3], lHJC_global[0:3], midHJC_global[0:3],
                               rKJC_global[0:3], lKJC_global[0:3]), axis=0)

np.savetxt("vin.csv", out, delimiter=",")

-199.8718021802319
-199.8677828844294
-199.8643109259956
-199.86098903704325
-199.85866790080024
-199.8579132969332
-199.85689010738434
-199.85774321081283
-199.86003644839505
-199.86334759759674
-199.86744890195607
-199.87197614070732
-199.87736276980053
-199.88317352043433
-199.88884149207112
-199.8944005088465
-199.90016350504638
-199.9050616701946
-199.90975879303153
-199.91342112364498
-199.91593893153066
-199.91782085017294
-199.91802523696873
-199.9178813431742
-199.91707211114044
-199.91489085799097
-199.9113460815642
-199.90801693251478
-199.902678834878
-199.89662484011265
-199.8894732393066
-199.88269192480948
-199.8751948496269
-199.8683061175248
-199.86018236963451
-199.853736847603
-199.84643198940572
-199.8398964294176
-199.83306657323755
-199.8267434338434
-199.8211342409665
-199.8156459191478
-199.81069098597357
-199.80612361213926
-199.80291588642237
-199.8001519305908
-199.79771441585635
-199.79647779048742
-199.79595983601843
-199.7949232517699
-199.795029624803
-19

In [ ]:
midAsis = np.array([0.00696365,0.0103228,0.0])
misPsis = np.array([-0.153155,0.0426324,0.0])

midPelvis = np.mean((midAsis, misPsis), axis=0)

#R1Meta = np.array([0.174285,-0.0147377,-0.0298436])
#R5Meta = np.array([0.137032,-0.0147377,0.0412118])
#RmidMeta = np.mean((R1Meta, R5Meta), axis=0)

print(midPelvis)
#print(RmidMeta)